In [1]:
import numpy as np
import pandas as pd
import os
import sys
from google.colab import drive
import matplotlib.pyplot as plt
import math
import itertools
import random
import seaborn as sns
# import package
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import ward, fcluster, leaders
from scipy.cluster import hierarchy
from scipy.sparse import csr_matrix
import networkx as nx
import seaborn as sns
import math

sc = StandardScaler()

sns.set_style("ticks")

sns.despine()

<Figure size 432x288 with 0 Axes>

In [2]:
#mount google colab
drive.mount('/content/drive',force_remount = True)

Mounted at /content/drive


In [3]:

# import source data

# source data
influenceFile = "/content/drive/MyDrive/Richard_FRL/Project Data/2021_ICM_Problem_D_Data/influence_data.csv"
fullFileRes = "/content/drive/MyDrive/Richard_FRL/Project Data/2021_ICM_Problem_D_Data/full_music_data.csv"
artistFile = "/content/drive/MyDrive/Richard_FRL/Project Data/2021_ICM_Problem_D_Data/data_by_artist.csv"

# read data
fullData = pd.read_csv(fullFileRes)
influenceData = pd.read_csv(influenceFile)
artistData = pd.read_csv(artistFile)


In [4]:
influenceData.columns

Index(['influencer_id', 'influencer_name', 'influencer_main_genre',
       'influencer_active_start', 'follower_id', 'follower_name',
       'follower_main_genre', 'follower_active_start'],
      dtype='object')

In [5]:
influenceData['influencer_active_start'].value_counts()

1960    12640
1970     7996
1950     6643
1980     6083
1990     3646
1940     2957
1930     2409
2000      376
2010       20
Name: influencer_active_start, dtype: int64

In [6]:
active_start = {'1930':1,'1940':2,'1950':3,'1960':4,'1970':5,'1980':6,'1990':7,'2000':8,'2010':9}

In [7]:

# const genre group

genre = {'Pop/Rock':0, 'R&B;':1, 'Country':2, 'Jazz':3, 'Vocal':4, 'Latin':5, 'Classical':6, 'Folk':7, 'International':8, 'Blues':9, 'Reggae':10, 'Electronic':11, 'Easy Listening':12, 'Stage & Screen':13, 'Religious': 14, 'Comedy/Spoken':15, 'New Age':16, 'Avant-Garde':17, "Children's": 18, 'Unknown': 19}

In [8]:
def cleanData(data):
    cleaned_data = data.drop_duplicates(keep='first',inplace=False).loc[:, ~data.columns.str.contains("^Unnamed")].dropna(axis=1,how='all').dropna(axis=0, how='any')
    return cleaned_data

In [9]:
# clean data

fullData = cleanData(fullData)
artistData = cleanData(artistData)


In [10]:
genre_res = pd.DataFrame(index=artistData.index, columns=['genre'])
active_start_res = pd.DataFrame(index=artistData.index, columns=['active_start_year'])

for row_index, row in artistData['artist_id'].items():
    _tem = (influenceData.loc[(influenceData['influencer_id'] == row) | (influenceData['follower_id'] == row)]['influencer_main_genre'])
    if _tem.shape[0] != 0:
        genre_res['genre'][row_index] = _tem.iloc[0]

for row_index, row in artistData['artist_id'].items():
    _tem = (influenceData.loc[(influenceData['influencer_id'] == row) | (influenceData['follower_id'] == row)]['influencer_active_start'])
    if _tem.shape[0] != 0:
        active_start_res['active_start_year'][row_index] = _tem.iloc[0]

artistData_concated = pd.concat([artistData,genre_res],axis=1)
artistData_concated = pd.concat([artistData_concated,active_start_res],axis=1)
#artistData_concated.to_csv('./artistData_concated.csv')

genre_res = pd.DataFrame(index=artistData.index, columns=['genre'])
active_start_res = pd.DataFrame(index=artistData.index, columns=['active_start_year'])

In [ ]:
for row_index, row in fullData['artists_id'].items():
    _tem = (influenceData.loc[(influenceData['influencer_id'] == eval(row)[0]) | (influenceData['follower_id'] == eval(row)[0])]['influencer_main_genre'])
    if _tem.shape[0] != 0:
        genre_res['genre'][row_index] = _tem.iloc[0]
        
fullData_concated = pd.concat([fullData,genre_res],axis=1)

In [ ]:
artistData = artistData_concated
fullData = fullData_concated

fullData = cleanData(fullData)
artistData = cleanData(artistData)

In [ ]:
artistData

In [ ]:
artistData['artist_name'].value_counts()

In [ ]:
artistData['popul_by_artist_genre'] = artistData.groupby(['artist_name','genre'])['popularity'].transform('mean') 


In [ ]:
artistData_plots = artistData[['genre','active_start_year','popul_by_artist_genre']]
artistData_plots['popul_by_genre'] = artistData_plots.groupby(['genre','active_start_year'])['popul_by_artist_genre'].transform('mean') 
artistData_plots

In [ ]:
fullData

INSIGHTS + 

In [ ]:
fullData['danceability'] = fullData['danceability'].astype('float')

plt.figure(figsize=(8, 6))


sns.kdeplot(
   data=fullData, x='danceability', hue='genre',
#    fill=True, common_norm=False, palette="bright",
#    alpha=.2, linewidth=0,
)
sns.despine()
plt.savefig('/content/sample_data/danceability.jpg', dpi=600)

plt.figure(figsize=(8, 6))

fullData['energy'] = fullData['energy'].astype('float')


sns.kdeplot(

   data=fullData, x='energy', hue='genre',
#     fill=True, common_norm=False, palette="crest",
#    alpha=.3, linewidth=0,
)
sns.despine()
plt.savefig('/content/sample_data/energy.jpg', dpi=600)

fullData['valence'] = fullData['valence'].astype('float')

plt.figure(figsize=(8, 6))

sns.kdeplot(
   data=fullData, x='valence', hue='genre',
#    fill=True, common_norm=False, palette="crest",
#    alpha=.3, linewidth=0,
)
sns.despine()
plt.savefig('/content/sample_data/valence.jpg', dpi=600)

fullData['tempo'] = fullData['tempo'].astype('float')

plt.figure(figsize=(8, 6))

sns.kdeplot(
   data=fullData, x='tempo', hue='genre',
#    fill=True, common_norm=False, palette="crest",
#    alpha=.3, linewidth=0,
)
sns.despine()
plt.savefig('/content/sample_data/tempo.jpg', dpi=600)


fullData['loudness'] = fullData['loudness'].astype('float')

plt.figure(figsize=(8, 6))

sns.kdeplot(
   data=fullData, x='loudness', hue='genre',
#    fill=True, common_norm=False, palette="crest",
#    alpha=.4, linewidth=0,
)
sns.despine()
plt.savefig('/content/sample_data/loudness.jpg', dpi=600)

fullData['acousticness'] = fullData['acousticness'].astype('float')

plt.figure(figsize=(8, 6))

sns.kdeplot(
   data=fullData, x='acousticness', hue='genre',
#    fill=True, common_norm=False, palette="crest",
#    alpha=.4, linewidth=0,
)
sns.despine()
plt.savefig('/content/sample_data/acousticness.jpg', dpi=600)

fullData['instrumentalness'] = fullData['instrumentalness'].astype('float')

plt.figure(figsize=(8, 6))

sns.kdeplot(
   data=fullData, x='instrumentalness', hue='genre',
#    fill=True, common_norm=False, palette="bright",
#    alpha=.4, linewidth=0,
)
sns.despine()
plt.savefig('/content/sample_data/instrumentalness.jpg', dpi=600)

fullData['liveness'] = fullData['liveness'].astype('float')

plt.figure(figsize=(8, 6))

sns.kdeplot(
   data=fullData, x='liveness', hue='genre',
#    fill=True, common_norm=False, palette="crest",
#    alpha=.4, linewidth=0,
)
sns.despine()
plt.savefig('/content/sample_data/liveness.jpg', dpi=600)

fullData['speechiness'] = fullData['speechiness'].astype('float')

plt.figure(figsize=(8, 6))

sns.kdeplot(
   data=fullData, x='speechiness', hue='genre',
#    fill=True, common_norm=False, palette="crest",
#    alpha=.4, linewidth=0,
)
sns.despine()
plt.savefig('/content/sample_data/speechiness.jpg', dpi=600)

fullData['duration_ms'] = fullData['duration_ms'].astype('float')

plt.figure(figsize=(8, 6))

sns.kdeplot(
   data=fullData, x='duration_ms', hue='genre',
#    fill=True, common_norm=False, palette="crest",
#    alpha=.4, linewidth=0,
)
sns.despine()
plt.savefig('/content/sample_data/duration_ms.jpg', dpi=600)

fullData['popularity'] = fullData['popularity'].astype('float')

plt.figure(figsize=(8, 6))

sns.kdeplot(
   data=fullData, x='popularity', hue='genre',
#    fill=True, common_norm=False, palette="crest",
#    alpha=.4, linewidth=0,
)
sns.despine()
plt.savefig('/content/sample_data/popularity.jpg', dpi=600)
